### Apply a newton-raphson optimization on rosenbrock’s valley function to find its minimum 
_In calculus, Newton's method is an iterative method for finding the roots of a differentiable function f (i.e. solutions to the equation f(x)=0). In optimization, Newton's method is applied to the derivative f ′ of a twice-differentiable function f to find the roots of the derivative (solutions to f ′(x)=0), also known as the stationary points of f._

In [2]:
# We're using Newton's method to find roots of f'(x) = 0
# Which gives us stationary points that can be used to find local optima

I used Wikipedia and:
- http://mathfaculty.fullerton.edu/mathews/n2003/newtonsearchmod.html
- http://www.robots.ox.ac.uk/~az/lectures/b1/lect2.pdf

In [3]:
a = 1
b = 100

In [4]:
# We're using Rosenbrock, so can manually compute partials
def f(x, y):
    return (a-x)**2 + b*(y-x**2)**2

def dx(x, y):
    return -4*b*x*(y-x**2)-2*(a-x)

def dxx(x, y):
    return 12*b*x**2 - 4*b*y +2

def dxy(x, y):
    return -4*b*x
    
def dy(x, y):
    return 2*b*(y-x**2)
    
def dyy(x, y):
    return 2*b

I will be using 2D arrays to represent matrices and vectors

In [5]:
# Manually compute Jacobian and Hessian of Rosenbrock
def J(x, y):
    return [[dx(x, y)], [dy(x,y)]] # i.e. 2x1

def H(x, y):
    return [[dxx(x, y), dxy(x, y)], [dxy(x, y), dyy(x, y)]] # i.e. 2x2

In [6]:
# We'll need to invert the Hessian, which is 2x2 -> simple inversion using determinant
def inverse(matrix):
    a = matrix[0][0]
    b = matrix[0][1]
    c = matrix[1][0]
    d = matrix[1][1]
    
    det = a*d - b*c
    return [[(1.0*d/det), (-1.0*b/det)], [(-1.0*c/det), 1.0*a/det]]

# We'll need to find a matrix product
def prod(H, J): # 2x2 * 2x1 = 2x1
    a = H[0][0]
    b = H[0][1]
    c = H[1][0]
    d = H[1][1]
    
    e = J[0][0]
    f = J[1][0]
    
    return [[1.0*a*e + 1.0*b*f], [1.0*c*e + 1.0*d*f]]

In [7]:
def NewtonSearch(x0, y0, iters):
    guess = [[x0], [y0]]
    for i in xrange(0, iters):
        print "P = ", (guess[0][0], guess[1][0]), "\nf(P) = ", f(guess[0][0], guess[1][0])
        
        hessian = H(guess[0][0], guess[1][0]) # 2x2
        jacobian = J(guess[0][0], guess[1][0]) # 2x1
        
        nextGuess = prod(inverse(hessian), jacobian) # 2x1
        
        # Element-wise subtraction
        guess = [[guess[0][0] - nextGuess[0][0]], [guess[1][0] - nextGuess[1][0]]]
        
    print "---------------"
    print "Minimised at: ", (guess[0][0], guess[1][0])
    return guess

In [8]:
NewtonSearch(0.1,0.3,10)

P =  (0.1, 0.3) 
f(P) =  9.22
P =  (0.08421052631578947, 0.006842105263157927) 
f(P) =  0.838676575533
P =  (0.9565060408276677, 0.15400434150227751) 
f(P) =  57.8986912531
P =  (0.9567899812218087, 0.9154469875442818) 
f(P) =  0.00186710572346
P =  (0.9999993032743315, 0.9981315610369098) 
f(P) =  0.000348585894962
P =  (0.999999810570859, 0.9999996211414965) 
f(P) =  3.58833994817e-14
P =  (1.0, 0.999999999999964) 
f(P) =  1.29392909979e-25
P =  (1.0, 1.0) 
f(P) =  0.0
P =  (1.0, 1.0) 
f(P) =  0.0
P =  (1.0, 1.0) 
f(P) =  0.0
---------------
Minimised at:  (1.0, 1.0)


[[1.0], [1.0]]

As we increase iterations, get closer to converge.  
**General solution (a, a^2) is noticable w/ < 10 iters**

### Problems and Improvements:
- We might jump over the minima  
_"Rather than jump straight to xn − H−1n gn, it is better to perform a line search which ensures global convergence"_  
**But this means we need more iterations**  
This seems to be a bad strategy in some of the cases I've run... Setting a default alpha = 1
- How do you know how many iterations to run for, i.e. when do you stop and say it's converged?  
Maybe compute a moving difference between successive results -> if we can see the function has increased, revert to previous -> but we may end up calling a premature solution -> stop if the function remains the same
- How do you come up with an initial guess?  
You'd want to plot the function and make an intuitive guess

In [9]:
# We define a weight alpha, a step size like in gradient descent
def NewtonSearch(x0, y0, iters, alpha=1):
    guess = [[x0], [y0]]
    for i in xrange(0, iters):
        print "P = ", (guess[0][0], guess[1][0]), "\nf(P) = ", f(guess[0][0], guess[1][0])
        
        hessian = H(guess[0][0], guess[1][0]) # 2x2
        jacobian = J(guess[0][0], guess[1][0]) # 2x1
        
        nextGuess = prod(inverse(hessian), jacobian) # 2x1
        
        # Element-wise subtraction
        guess = [[guess[0][0] - alpha*nextGuess[0][0]], [guess[1][0] - alpha*nextGuess[1][0]]]
        
    print "---------------"
    print "Minimised at: ", (guess[0][0], guess[1][0])
    return guess

In [12]:
NewtonSearch(0.0002, -10, 1000, 0.1)

P =  (0.0002, -10) 
f(P) =  10000.99968
P =  (0.0002499650172914942, -8.999999976013992) 
f(P) =  8100.99956943
P =  (0.00030547584631474586, -8.099999944412811) 
f(P) =  6561.99945026
P =  (0.00036714731574720514, -7.289999902961693) 
f(P) =  5315.40932089
P =  (0.00043566224703645555, -6.560999848875662) 
f(P) =  4305.67127962
P =  (0.0005117789338979385, -5.904899778685603) 
f(P) =  3487.78342566
P =  (0.0005963394352769157, -5.3144096880727085) 
f(P) =  2825.29421893
P =  (0.0006902787680558949, -4.7829685716639085) 
f(P) =  2288.67791147
P =  (0.0007946350944212813, -4.3046715227791275) 
f(P) =  1854.0186469
P =  (0.0009105610081739345, -3.874204123119122) 
f(P) =  1501.9445809
P =  (0.0010393360344089872, -3.4867833933800396) 
f(P) =  1216.76451894
P =  (0.0011823804679240708, -3.138104648677628) 
f(P) =  985.768592671
P =  (0.001341270687453948, -2.8242936682701236) 
f(P) =  798.661807906
P =  (0.001517756095369584, -2.541863648112997) 
f(P) =  647.105218431
P =  (0.001713777845

[[0.9999999999999996], [0.9999999999999996]]

In [13]:
# Even after 1000 iters, didn't converge to exact solution -> but this alpha rate would be useful for other cases